In [1]:
import requests
import httpx
from fake_useragent import UserAgent
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
from lxml import html
from latest_user_agents import get_latest_user_agents, get_random_user_agent
import psycopg2

In [2]:
def get_contrator_req(taxid):
#     ua = UserAgent()
#     user_agent = ua.random
    url = 'https://dauthau.asia/businesslistings/search/?q=' + taxid + '&industry1=&province=0&businesstype=0&lvkd=0'
    headers = {'User-Agent': get_random_user_agent()}
    scraper = cloudscraper.create_scraper()
#     client = httpx.Client(http2=True)
    response= scraper.get(url.strip(), headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    link = soup.find('div', class_= 'listing-summary').a['href']
    response= scraper.get('https://dauthau.asia' + link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find("table")
    df = pd.read_html(str(table), converters={'Mã số thuế': str, 'Điện thoại công ty': str})[0]
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df.iloc[1:2]
    df = df.iloc[:,:17]
    return df

In [3]:
def get_contractor_list(tax_id_lst):
    df = get_contrator_req(tax_id_lst[0])
    for i in tax_id_lst:
        if i != tax_id_lst[0]:
            try:
                df = pd.concat([df,get_contrator_req(i)])
                print(i)
            except:
                continue
        else:
            pass
    return df

In [4]:
conn = psycopg2.connect(host="localhost",database="postgres", user="postgres", password="dang1199")

In [5]:
sql = """select distinct "Mã số thuế" from kq_dauthau_202210 kd 
        where "Mã số thuế" not in (select "TAX_ID" from contractor_info ci)"""

In [6]:
input_file = pd.read_sql(sql, conn)
u_lst = list(set(input_file["Mã số thuế"].tolist()))
len(u_lst)

/home/dangpdh/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


2173

In [7]:
df = get_contractor_list(u_lst)

0901116881
2801296049
0101527931
0104394831
0400383773
6001392086
0401679300
0108790579
0101998585
1801327961
3801155025
0106338254
6001690357
4300333279
0315538962
0103250643
0600664309
5500402690
0102385623
4400355572
0315745990
0100150295
0103210753
0101452612
0401772356
0314264042
2600690239
0301458065
2500626127
3800672863
0300588569
2801741853
0303716473
5901106762
0100148391
0104354620
4100142502
5600333747
3702553642
0109988035
3500771422
0102393166
0314373468
0107335168
2400909530
0107729807
0101520284
1601011858
2500217389
0106512696
3900370376
0302118865
0401810499
4300752551
6001718965
2500641781
0107455786
0108863650
0106668492
3101030221
0316383817
4200589245
3801169807
0108168720
1201601186
0109429326
5300700481
2301131680
0103494657
0304865661
0106199882
1201582494


,Tên công ty,Tên quốc tế,Lượt xem,Mã số thuế,Quốc gia cấp MST,Mã định danh,Địa chỉ giao dịch,Địa chỉ trụ sở,Điện thoại công ty,Fax công ty,...,Ngày thành lập công ty,Vốn điều lệ,Số nhân viên,Ngày phê duyệt trên mạng đấu thầu quốc gia,Ngày cấp MST,Trạng thái nộp phí,Trạng thái công ty,Cập nhật lần cuối,Người đại diện pháp luật,Chức vụ
1,CÔNG TY CỔ PHẦN XÂY LẮP VÀ PHÁT TRIỂN ĐIỆN LỰC...,CED POWER DEVELOPMENT AND CONSTRUCTION JOINT S...,197,0107705436,VN,vn0107705436,"N07-Khu giãn dân Hà Trì, Phường Hà Cầu, Quận H...","N07-Khu Giãn Dân Hà Trì, Phường Hà Cầu, Quận H...",0976195896,NaN,...,16/01/2017,NaN,32,14/04/2022,NaN,NaN,NaN,NaN,NaN,NaN
1,CÔNG TY TNHH ĐẠI PHÚC HƯNG YÊN,DAI PHUC HUNG YEN COMPANY LIMITED,28,0901116881,VN,vn0901116881,"Đường La Tiến, Xã Tống Phan, Huyện Phù Cừ, Tỉn...","Đội 12, Thôn Duyệt Lễ, Xã Minh Tân, Huyện Phù ...",0866802368,NaN,...,22/02/2022,NaN,NaN,10/06/2022,21/02/2022,NaN,NaN,NaN,NaN,NaN
1,HỘ KINH DOANH NGUYỄN THỊ LIÊN,NaN,10,2801296049,VN,vn2801296049,"Phố 3, thị trấn Cành Nàng, huyện Bá Thước, tỉn...","Phố 3, Thị Trấn Cành Nàng, Huyện Bá Thước, Tỉn...",0913349497,NaN,...,28/07/2008,NaN,NaN,09/09/2022,NaN,"Đã nộp chi phí Theo Thông tư 06/2021, từ tháng...",NaN,NaN,NaN,NaN
1,CÔNG TY TNHH MÁY VÀ THIẾT BỊ THUẬN THÀNH,THUAN THANH EQUIPMENT AND MACHINE COMPANY LIMITED,239,0101527931,VN,vn0101527931,NaN,"Số Nhà 154, Đường Hoàng Mai, Phường Hoàng Văn ...",0462919148,0436627698,...,23/07/2014,NaN,27,15/04/2022,23/07/2004,NaN,NaN,NaN,NaN,NaN
1,CHI NHÁNH XÍ NGHIỆP TÀI NGUYÊN VÀ MÔI TRƯỜNG 7...,Enterprise of Natural Resources and Environment 7,225,0104394831008,NaN,NaN,NaN,"135 Đường Thành Thái, Phường Đông Thọ, Thành p...",02373852048,02373851191,...,08/06/1999,NaN,80,23/09/2019,NaN,Tài khoản chưa nộp phí duy trì. Sau ngày 31/03...,Đang hoạt động,10:17 10/08/2022 - Cập nhật lần thứ 3. Vui lòn...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,CÔNG TY TNHH TƯ VẤN ĐẦU TƯ XD & TM LUY LÂU,NaN,40,2301131680,VN,vn2301131680,"Thôn Lũng Khê, Xã Thanh Khương, Huyện Thuận Th...","Thôn Lũng Khê, Xã Thanh Khương, Huyện Thuận Th...",0965325666,NaN,...,14/04/2020,NaN,NaN,14/04/2022,NaN,Tài khoản chưa nộp phí duy trì. Sau ngày 31/03...,NaN,NaN,NaN,NaN
1,CÔNG TY CỔ PHẦN 568 VIỆT NAM,"568 VIETNAM.,JSC",270,0103494657,VN,vn0103494657,NaN,Lk18 L12 Khu Đấu Giá Qsd Đất Mậu Lương Ph Kiến...,02463261787,NaN,...,05/03/2009,NaN,50,03/05/2018,NaN,"Đã nộp chi phí Theo Thông tư 06/2021, từ tháng...",NaN,NaN,NaN,NaN
1,CÔNG TY TNHH XÂY DỰNG LÊ PHƯƠNG,NaN,322,0304865661,VN,vn0304865661,NaN,"191 Đường Số 5, Phường Phước Bình, Quận 9, Thà...",0913752181,NaN,...,10/02/2007,NaN,100,14/04/2022,NaN,"Đã nộp chi phí Theo Thông tư 06/2021, từ tháng...",NaN,NaN,NaN,NaN
1,CÔNG TY TNHH DỊCH VỤ DU LỊCH ĐƯỜNG SẮT SÔNG HỒNG,SONG HONG RAILWAY TOURIST SERVICES COMPANY LIM...,11,0106199882,VN,vn0106199882,"Phòng 303, Số 2D, phố Khâm Thiên, Phường Khâm ...","Phòng 303, Số 2D, Phố Khâm Thiên, Phường Khâm ...",0906295725,NaN,...,06/06/2013,NaN,NaN,09/09/2022,NaN,"Đã nộp chi phí Theo Thông tư 06/2021, từ tháng...",NaN,NaN,NaN,NaN


In [15]:
u_lst

['2100664618',
 '1704000041',
 '0311521083',
 '5300200993',
 'vn0102021143',
 '0401780117',
 '2300195025006',
 '0102306702',
 '0109141986',
 '6001135836',
 'vn0200164830',
 '0106899524',
 '0106090028',
 '0100687474',
 '0100122379',
 '5901145987',
 '2500379774',
 '0107307146',
 '0314360853',
 '0102385623',
 '3002142397',
 '3502208399',
 '1602024786',
 '3602491569',
 'vn0307758548',
 '2901739230',
 '2901608125',
 '0106250578',
 '0102761363',
 '2802826919',
 '3600252847',
 '0310140399',
 '0312192201',
 '3100137028',
 '2100597954',
 'vn0302361203',
 '1902001001',
 '0100510533',
 'vn0102230877',
 '0107593810',
 '2802368881',
 '0100774631',
 '0106031505',
 'vn4600432224',
 '4600972251',
 '5200134722',
 '0104606490',
 '0100005',
 '3002142774',
 '0302564475',
 '3801216856',
 'vn2500609435',
 '5900376205',
 '0108213405',
 '4300721592',
 '5600179100',
 '0108228112',
 '0310337275',
 '43GP/KDBH',
 '4601501456',
 '0108596243',
 'vn0500142845',
 '2802472931',
 '0500297165',
 '0105880908',
 '01802388